In [4]:
import numpy as np
import tensorflow as tf
import util
import numpy as np
import tensorflow as tf
import util
import torch
from keras.utils.np_utils import *
from keras.datasets import mnist
MNIST_CLASSIFIER_FROZEN_GRAPH ='/home/imi432_006/huangdengrong/GAN-Research/metrics_gan/metrics/classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def calculate_labels(images,epoch,global_step):
    label_dict={}
    for i in range(10):
        label_dict[i]=0
    eval_images = tf.convert_to_tensor(images)
    y_logit = util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    labels=tf.argmax(y_logit, 1)
    labels=tf.Session().run(labels)
    for data in labels:
        label_dict[data]+=1
    for i in range(10):
        label_dict[i]=label_dict[i]/len(images)
    max_value=max(label_dict.values())
    min_value=min(label_dict.values())
    print('epoch:%d   global_step:%d'%(epoch,global_step))
    print(label_dict)
    print('chazhi:%.8f'%(max_value-min_value))
    return label_dict,max_value-min_value

In [6]:

# Large amount of credit goes to:
# https://github.com/keras-team/keras-contrib/blob/master/examples/improved_wgan.py
# which I've used as a reference for this implementation

from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np
import os
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class WGANGP():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and critic
        self.generator = self.build_generator()
        self.critic = self.build_critic()

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        real_img = Input(shape=self.img_shape)

        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_img = self.generator(z_disc)

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, z_disc],
                            outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                              self.wasserstein_loss,
                                              partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(100,))
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)


    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)


    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0
        steps=[]
        values=[]

        for epoch in range(epochs):

            for index in range(nb_batches):
                for _ in range(self.n_critic):
                    global_step += 1
                    imgs = X_train[index * batch_size:(index + 1) * batch_size]
                    # Sample generator input
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                    # Train the critic
                    d_loss = self.critic_model.train_on_batch([imgs, noise],
                                                              [valid, fake, dummy])

                    # ---------------------
                    #  Train Generator
                    # ---------------------

                g_loss = self.generator_model.train_on_batch(noise, valid)

                # Plot the progress
#                 print("epoch%d step%d [D loss: %f] [G loss: %f]" % (epoch,global_step, d_loss[0], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    label_dict,value=self.mode_drop(epoch,global_step)
                    steps.append(global_step)
                    values.append(value)
#             plt.subplots(1, 1)
        plt.plot(steps,values)
        plt.xlim([0,40000])
        plt.ylim([0.0,1.0])
        plt.xlabel('steps')
        plt.ylabel('epochs')
        plt.tick_params(axis='both',which='major',labelsize=13)
        plt.legend(loc='lower right')
        if not os.path.isdir('images_wgan'):
            os.mkdir('images_wgan')
        plt.savefig("images_wgan/mode_drop.png")
        plt.show()


    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cnt += 1
        if not os.path.isdir('images_wgan'):
            os.mkdir('images_wgan')
        fig.savefig("images_wgan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()
    def mode_drop(self,epoch,global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        label_dict,value=calculate_labels(gen_imgs,epoch,global_step)
        return label_dict,value


if __name__ == '__main__':
    wgan = WGANGP()
    wgan.train(epochs=20, batch_size=32, sample_interval=200)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_5 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_9 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
batch_normalization_21 (Batc (None, 14, 14, 128)       512       
_________________________________________________________________
activation_13 (Activation)   (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_10 (UpSampling (None, 28, 28, 128)       0         
__________

epoch:0   global_step:4800
{0: 0.1436, 1: 0.0274, 2: 0.0777, 3: 0.1286, 4: 0.1467, 5: 0.0266, 6: 0.0235, 7: 0.17, 8: 0.2262, 9: 0.0297}
chazhi:0.20270000
epoch:0   global_step:5000
{0: 0.0863, 1: 0.021, 2: 0.072, 3: 0.1629, 4: 0.1492, 5: 0.0312, 6: 0.0228, 7: 0.1733, 8: 0.2408, 9: 0.0405}
chazhi:0.21980000
epoch:0   global_step:5200
{0: 0.0795, 1: 0.0284, 2: 0.0857, 3: 0.1555, 4: 0.1637, 5: 0.0303, 6: 0.032, 7: 0.16, 8: 0.2301, 9: 0.0348}
chazhi:0.20170000
epoch:0   global_step:5400
{0: 0.0943, 1: 0.0268, 2: 0.0781, 3: 0.1458, 4: 0.1756, 5: 0.0217, 6: 0.0295, 7: 0.1839, 8: 0.2114, 9: 0.0329}
chazhi:0.18970000
epoch:0   global_step:5600
{0: 0.1, 1: 0.0216, 2: 0.0878, 3: 0.1254, 4: 0.1862, 5: 0.0218, 6: 0.0263, 7: 0.1844, 8: 0.2166, 9: 0.0299}
chazhi:0.19500000
epoch:0   global_step:5800
{0: 0.1296, 1: 0.0124, 2: 0.0878, 3: 0.1346, 4: 0.1333, 5: 0.0231, 6: 0.0361, 7: 0.1351, 8: 0.2571, 9: 0.0509}
chazhi:0.24470000
epoch:0   global_step:6000
{0: 0.1003, 1: 0.0169, 2: 0.0776, 3: 0.169, 4: 

epoch:1   global_step:15400
{0: 0.1302, 1: 0.0758, 2: 0.0764, 3: 0.101, 4: 0.1597, 5: 0.0276, 6: 0.0661, 7: 0.1288, 8: 0.1403, 9: 0.0941}
chazhi:0.13210000
epoch:1   global_step:15600
{0: 0.1222, 1: 0.1076, 2: 0.0676, 3: 0.0892, 4: 0.1879, 5: 0.0195, 6: 0.0694, 7: 0.1433, 8: 0.1221, 9: 0.0712}
chazhi:0.16840000
epoch:1   global_step:15800
{0: 0.1376, 1: 0.083, 2: 0.0707, 3: 0.1009, 4: 0.1739, 5: 0.0321, 6: 0.0633, 7: 0.152, 8: 0.121, 9: 0.0655}
chazhi:0.14180000
epoch:1   global_step:16000
{0: 0.1316, 1: 0.0785, 2: 0.0762, 3: 0.0801, 4: 0.1884, 5: 0.0255, 6: 0.0831, 7: 0.1261, 8: 0.138, 9: 0.0725}
chazhi:0.16290000
epoch:1   global_step:16200
{0: 0.1091, 1: 0.091, 2: 0.0729, 3: 0.0928, 4: 0.1372, 5: 0.0268, 6: 0.0812, 7: 0.1611, 8: 0.1558, 9: 0.0721}
chazhi:0.13430000
epoch:1   global_step:16400
{0: 0.0978, 1: 0.0796, 2: 0.0757, 3: 0.0981, 4: 0.1529, 5: 0.0311, 6: 0.067, 7: 0.1801, 8: 0.1441, 9: 0.0736}
chazhi:0.14900000
epoch:1   global_step:16600
{0: 0.1017, 1: 0.0596, 2: 0.0723, 3: 

KeyboardInterrupt: 